In [ ]:
pip install openpyxl

In [ ]:
pip install mlxtend

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv(r"C:\Users\Lilian\Downloads\online_retail_II.csv\online_retail_II.csv")

In [3]:
# --- Data Cleaning ---
# Remove extra spaces from the description
df['Description'] = df['Description'].str.strip()
# Drop rows with no InvoiceNo
df.dropna(axis=0, subset=['Invoice'], inplace=True)
# Convert InvoiceNo to string
df['Invoice'] = df['Invoice'].astype('str')
# Remove credit transactions (invoices starting with 'C')
df = df[~df['Invoice'].str.contains('C')]

In [4]:
# For this analysis, let's focus on the UK market
df_uk = df[df['Country'] == 'United Kingdom']

print("Data is loaded and cleaned.")
df_uk.head()

Data is loaded and cleaned.


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
# Group by invoice number and description, summing quantities
# This creates a sparse matrix-like structure
basket = (df_uk.groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

# We don't care about the quantity, just whether the item was purchased or not.
# So, we'll convert any positive number to 1 and keep 0s as 0s.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

# Drop the 'POSTAGE' column as it's not a product we want to analyze
if 'POSTAGE' in basket_sets.columns:
    basket_sets.drop('POSTAGE', inplace=True, axis=1)

print("Data is now in the correct one-hot encoded format.")
basket_sets.head()

C:\Users\Lilian\AppData\Local\Temp\ipykernel_12996\3599498955.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)


Data is now in the correct one-hot encoded format.


Description,*Boombox Ipod Classic,*USB Office Glitter Lamp,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,11 PC CERAMIC TEA SET POLKADOT,12 ASS ZINC CHRISTMAS DECORATIONS,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Generate the frequent itemsets
# We'll use a min_support of 0.05 (i.e., the itemset appears in at least 5% of all transactions)
# Note: You may need to adjust this value. If you get no results, lower it. If you get too many, raise it.
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True, low_memory=True)

# Check if we found any itemsets before proceeding
if frequent_itemsets.empty:
    print("Still no frequent itemsets found. Try lowering min_support even more (e.g., 0.005).")
else:
    # Sort by support value to see the most popular itemsets
    frequent_itemsets.sort_values('support', ascending=False, inplace=True)
    print("Success! Found frequent itemsets. Here are the top 10:")
    print(frequent_itemsets.head(10))

C:\Users\Lilian\anaconda3\envs\py13\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Success! Found frequent itemsets. Here are the top 10:
      support                              itemsets
629  0.140093  (WHITE HANGING HEART T-LIGHT HOLDER)
464  0.091716            (REGENCY CAKESTAND 3 TIER)
272  0.081294             (JUMBO BAG RED RETROSPOT)
36   0.070579       (ASSORTED COLOUR BIRD ORNAMENT)
387  0.067594                       (PARTY BUNTING)
307  0.060025             (LUNCH BAG  BLACK SKULL.)
285  0.059411              (JUMBO STORAGE BAG SUKI)
574  0.057279      (STRAWBERRY CERAMIC TRINKET BOX)
283  0.056186   (JUMBO SHOPPER VINTAGE RED PAISLEY)
240  0.056026               (HEART OF WICKER SMALL)


In [10]:
# Generate the rules from our frequent itemsets
# We are looking for rules with a lift greater than 1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values(['lift', 'confidence'], ascending=[False, False], inplace=True)

print("\nTop Association Rules Found:")
print(rules.head(10))


Top Association Rules Found:
                              antecedents  \
648        (POPPY'S PLAYHOUSE LIVINGROOM)   
649           (POPPY'S PLAYHOUSE BEDROOM)   
578        (POPPY'S PLAYHOUSE LIVINGROOM)   
579           (POPPY'S PLAYHOUSE KITCHEN)   
304           (POPPY'S PLAYHOUSE BEDROOM)   
305           (POPPY'S PLAYHOUSE KITCHEN)   
616  (WOODEN STAR CHRISTMAS SCANDINAVIAN)   
617  (WOODEN TREE CHRISTMAS SCANDINAVIAN)   
444        (SMALL MARSHMALLOWS PINK BOWL)   
445  (SMALL DOLLY MIX DESIGN ORANGE BOWL)   

                              consequents  antecedent support  \
648           (POPPY'S PLAYHOUSE BEDROOM)            0.013194   
649        (POPPY'S PLAYHOUSE LIVINGROOM)            0.016419   
578           (POPPY'S PLAYHOUSE KITCHEN)            0.013194   
579        (POPPY'S PLAYHOUSE LIVINGROOM)            0.017432   
304           (POPPY'S PLAYHOUSE KITCHEN)            0.016419   
305           (POPPY'S PLAYHOUSE BEDROOM)            0.017432   
616  (WOODEN TREE C